In [1]:
import csv
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
import sys
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=50000
## new try with 100 spreaders
N_patient_zero = 20;
lamb_load = 0.03;
lamb = 0.05
mu = 0.05;
scale=1.0; # Easy Case
T=100;
seed=2
location="networks"
# SIR parameters
t1 = 10;
#seed=int();

In [2]:
## import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS
log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb_load,seed)
csv_file=location+"/"+log_fname
print("seed = %d"%seed, flush=True)
np.random.seed(seed);
if path.exists(csv_file): print("Loading model from %s"%csv_file)
else:
    print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
    sys.exit()
print("Load Proximity model", flush=True)
tic = time()
model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
model.load_transmissions(csv_file, new_lambda = lamb)
model.recover_probas = mu*np.ones(N)
print(f"Loading took {time()-tic:.1f}s",flush=True)
model.initial_states = patient_zeros_states(N, N_patient_zero)
model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
t_max = len(model.transmissions)
print("Save plain dynamics", flush=True)
db = pd.DataFrame()
db["S"] = np.sum(model.states==0,axis=1)
db["I"] = np.sum(model.states==1,axis=1)
db["R"] = np.sum(model.states==2,axis=1)
db.to_csv("csv/Proximity_N%dK_T%d_s1_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,N_patient_zero,mu,lamb,seed),
          index=False, sep="\t")
del db
model.initial_states = model.states[t1]
model.states = model.states[t1:]
model.transmissions = model.transmissions[t1:]

seed = 2
Loading model from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s2.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s2.csv
Loading took 31.3s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics


In [3]:
# trac parameters
trac_tau = 5;
# MF parameters
MF_tau = 5;
MF_delta = 15;
# observation parameters
n_ranking = 50
p_untracked=0
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################

intervention_options=dict(quarantine_time=T-t1)
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)

In [4]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
##### RANDOM SCENARIO #######
scenario_rnd = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["random"]),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_rnd.run(t_max-t1,  print_every = 1)
print("Save random strategy", flush=True)
scenario_rnd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_rnd.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking),
          index=False, sep="\t")
del scenario_rnd

Using ranking_random to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49701, I:261, R:38 I+R:299
obser: S:50, I:0, R:0, I+R:0
free birds: 261
obs_symp:0(0), obs_rank:0(0)
t = 1 / 90
state: S:49649, I:306, R:45 I+R:351
obser: S:100, I:0, R:0, I+R:0
free birds: 306
obs_symp:0(0), obs_rank:0(0)
t = 2 / 90
state: S:49583, I:355, R:62 I+R:417
obser: S:149, I:1, R:0, I+R:1
free birds: 354
obs_symp:0(0), obs_rank:1(1)
t = 3 / 90
state: S:49504, I:416, R:80 I+R:496
obser: S:198, I:1, R:1, I+R:2
free birds: 415
obs_symp:0(0), obs_rank:0(1)
t = 4 / 90
state: S:49416, I:485, R:99 I+R:584
obser: S:248, I:1, R:1, I+R:2
free birds: 484
obs_symp:0(0), obs_rank:0(1)
t = 5 / 90
state: S:49338, I:525, R:137 I+R:662
obser: S:297, I:2, R:1, I+R:3
free birds: 523
obs_symp:0(0), obs_rank:1(2)
t = 6 / 90
state: S:49223, I:612, R:165 I+R:777
obser: S:347, I:21, R:8, I+R:29
free birds: 591
obs_symp:19(19), obs_rank:0(2)
t = 7 / 90
state: S:49134, I:674, R:192 I+R:866
obser: S:39

In [5]:
##### TRACING SCENARIO #######
scenario_trac = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing"], tau=5),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac.run(t_max-t1, print_every = 1)
print("Save tracing strategy", flush=True)
scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac_t%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau),
                  index=False, sep="\t")
del scenario_trac

Using ranking_tracing to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49701, I:261, R:38 I+R:299
obser: S:50, I:0, R:0, I+R:0
free birds: 261
obs_symp:0(0), obs_rank:0(0)
t = 1 / 90
state: S:49649, I:306, R:45 I+R:351
obser: S:100, I:0, R:0, I+R:0
free birds: 306
obs_symp:0(0), obs_rank:0(0)
t = 2 / 90
state: S:49583, I:355, R:62 I+R:417
obser: S:149, I:1, R:0, I+R:1
free birds: 354
obs_symp:0(0), obs_rank:1(1)
t = 3 / 90
state: S:49504, I:416, R:80 I+R:496
obser: S:198, I:1, R:1, I+R:2
free birds: 415
obs_symp:0(0), obs_rank:0(1)
t = 4 / 90
state: S:49416, I:485, R:99 I+R:584
obser: S:248, I:1, R:1, I+R:2
free birds: 484
obs_symp:0(0), obs_rank:0(1)
t = 5 / 90
state: S:49338, I:525, R:137 I+R:662
obser: S:297, I:2, R:1, I+R:3
free birds: 523
obs_symp:0(0), obs_rank:1(2)
t = 6 / 90
state: S:49253, I:584, R:163 I+R:747
obser: S:347, I:19, R:10, I+R:29
free birds: 565
obs_symp:17(17), obs_rank:0(2)
t = 7 / 90
state: S:49144, I:656, R:200 I+R:856
obser: S:

In [6]:
##### MF SCENARIO #######
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import ranking
imp.reload(ranking)
from ranking import RANKINGS
scenario_MF = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["backtrack"], 
                         algo="MF", init="all_S", tau=5, delta=10), 
    observation_options=observation_options,
    intervention_options=intervention_options,
)
scenario_MF.run(t_max-t1, print_every = 1)
print("Save MF strategy", flush=True)
scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta),
                  index=False, sep="\t")
del scenario_MF
# 1h01min per round
print("End seed", flush=True)

Using ranking_backtrack to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49701, I:261, R:38 I+R:299
obser: S:50, I:0, R:0, I+R:0
free birds: 261
obs_symp:0(0), obs_rank:0(0)
t = 1 / 90
state: S:49634, I:324, R:42 I+R:366
obser: S:100, I:0, R:0, I+R:0
free birds: 324
obs_symp:0(0), obs_rank:0(0)
t = 2 / 90
state: S:49553, I:394, R:53 I+R:447
obser: S:149, I:1, R:0, I+R:1
free birds: 393
obs_symp:0(0), obs_rank:1(1)
t = 3 / 90
state: S:49458, I:471, R:71 I+R:542
obser: S:195, I:5, R:0, I+R:5
free birds: 466
obs_symp:0(0), obs_rank:4(5)
t = 4 / 90
state: S:49365, I:545, R:90 I+R:635
obser: S:228, I:19, R:3, I+R:22
free birds: 526
obs_symp:0(0), obs_rank:14(19)
t = 5 / 90
state: S:49282, I:604, R:114 I+R:718
obser: S:260, I:33, R:7, I+R:40
free birds: 572
obs_symp:0(0), obs_rank:14(33)
t = 6 / 90
state: S:49181, I:671, R:148 I+R:819
obser: S:303, I:63, R:15, I+R:78
free birds: 613
obs_symp:25(25), obs_rank:7(40)
t = 7 / 90
state: S:49097, I:724, R:179 I+R:90

In [7]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu)
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=150/N
psus = 0.60
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                            tau=5
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


AttributeError: module 'sib' has no attribute 'PriorDiscrete'